In [16]:
import os
import re
import lpips
import torch
from PIL import Image
import torchvision.transforms as T

# 1) Init LPIPS model
loss_fn = lpips.LPIPS(net='alex').cuda()  # move to GPU if available
loss_fn.eval()

# 2) Pre‐processing to [-1,1]
transform = T.Compose([
    T.Resize((512, 512)),            # or your working resolution
    T.ToTensor(),
    T.Normalize((0.5,)*3, (0.5,)*3),  # LPIPS expects inputs in [-1,1]
])

def load_tensor(path):
    img = Image.open(path).convert('RGB')
    t = transform(img).unsqueeze(0)
    return t.cuda() if torch.cuda.is_available() else t

def average_lpips_from_folder(folder):
    # regex to pull off the "<base>_cs.ext"
    pattern = re.compile(r'(.+)_cs\.(png|jpg|jpeg|bmp)$', re.IGNORECASE)
    scores_stylepreservation = []
    scores_contentpreservation = []

    for fname in sorted(os.listdir(folder)):
        m = pattern.match(fname)
        if not m:
            continue
        base, _ = m.groups()
        stylized_path = os.path.join(folder, fname)
        content_path  = os.path.join(folder, f"{base}_c.{fname.split('.')[-1]}")
        style_path  = os.path.join(folder, f"{base}_s.{fname.split('.')[-1]}")

        if not os.path.isfile(content_path):
            print(f"  → skipping {base}: no content file found")
            continue

        # load
        stylized = load_tensor(stylized_path)
        content  = load_tensor(content_path)
        style = load_tensor(style_path)

        # compute LPIPS
        with torch.no_grad():
            score_style = loss_fn(stylized, style).item()
            score_content = loss_fn(stylized, content).item()
        scores_stylepreservation.append(score_style)
        scores_contentpreservation.append(score_content)
        #print(score_content)

    if not scores_stylepreservation:
        print("No valid _cs/_c pairs found in folder.")
        return

    print("\nSTYLE PRESERVATION")
    avg = sum(scores_stylepreservation) / len(scores_stylepreservation)
    print(f"Computed LPIPS on {len(scores_stylepreservation)} pairs.  Average LPIPS = {avg:.4f}")
    print("\nCONTENT PRESERVATION")
    avg = sum(scores_contentpreservation) / len(scores_contentpreservation)
    print(f"Computed LPIPS on {len(scores_contentpreservation)} pairs.  Average LPIPS = {avg:.4f}")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/cv27f25/.local/lib/python3.11/site-packages/lpips/weights/v0.1/alex.pth


In [17]:
if __name__ == "__main__":
    data_folder = "../AdaAttN/results/AdaAttN_retrain/test_latest/images"
    average_lpips_from_folder(data_folder)


STYLE PRESERVATION
Computed LPIPS on 20250 pairs.  Average LPIPS = 0.6833

CONTENT PRESERVATION
Computed LPIPS on 20250 pairs.  Average LPIPS = 0.5480


In [18]:
if __name__ == "__main__":
    data_folder = "../AdaAttN/results/AdaAttN/test_latest/images"
    average_lpips_from_folder(data_folder)


STYLE PRESERVATION
Computed LPIPS on 20250 pairs.  Average LPIPS = 0.6811

CONTENT PRESERVATION
Computed LPIPS on 20250 pairs.  Average LPIPS = 0.5378
